## Building A Chatbot
Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This code will cover the basics.

In [3]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
print("done")



done


In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000247B779CB90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000247B779F290>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hello Krish, it's nice to meet you!\n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on?  I'm always eager to learn more about the exciting advancements in the field of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 22, 'total_tokens': 76, 'completion_time': 0.098181818, 'prompt_time': 0.002119319, 'queue_time': 0.017759279, 'total_time': 0.100301137}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1366a066-349f-45c2-a923-cbc3befed1d3-0', usage_metadata={'input_tokens': 22, 'output_tokens': 54, 'total_tokens': 76})

In [5]:
# one naive way to make conversation history By passing a sequence of messages
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Krish, and according to our conversation, you are a Chief AI Engineer.  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm here to listen and learn! 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 97, 'total_tokens': 148, 'completion_time': 0.092727273, 'prompt_time': 0.004278908, 'queue_time': 0.021261210000000003, 'total_time': 0.097006181}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-efcb8536-2a1b-4ac9-9fdc-e401e4c4ecd4-0', usage_metadata={'input_tokens': 97, 'output_tokens': 51, 'total_tokens': 148})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

1.	ChatMessageHistory is a simple in-memory (temporary) for chat history
2.	BaseChatMessageHistory is an abstract base class (ABC) that defines the structure for chat message history 
3. RunnableWithMessageHistory It’s task is to add memory to runnables to create stateful applications  

##### A function called get_session_history is created to manage chat history for different sessions.
- Each session is identified by a unique session ID and the chat history is stored in a dictionary (store) where the key is the session ID.
- When the session ID is changed, the chatbot no longer remembers the previous conversation, highlighting the importance of session management.

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history) # model + history

##### This is how BaseChatMessageHistory can read session ID so format is important

In [7]:
config={"configurable":{"session_id":"chat1"}}

Above RunnableWithMessageHistory only take model and history. 
- since we are using message structure (HumanMessage) that is why it does not require put input keys. 
- we dont have prompt so no need of chat history key. 
- Output from invoke is also a structured message (AIMessage)  that is why it does not require put output keys.

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="explain features of controller in points")],
    config=config
)

In [14]:
response.content

'Here are some key features of a controller, presented in points:\n\n**General:**\n\n* **User Interface:**  Provides physical buttons, joysticks, triggers, and other inputs for interacting with a system (e.g., game console, computer, robot).\n* **Input Mechanism:**  Converts user actions into electrical signals that can be interpreted by the target system.\n\n* **Connectivity:**  Establishes a connection to the target system via wired or wireless protocols (e.g., USB, Bluetooth, Wi-Fi).\n* **Ergonomics:**  Designed for comfortable grip and handling, often considering hand size and play style.\n\n**Specific Features:**\n\n* **Buttons:**  Offer distinct tactile feedback for selecting options, triggering actions, and navigating menus.\n\n* **Joysticks:**  Allow for precise directional input and analog control (e.g., steering a vehicle or aiming a weapon).\n\n* **Triggers:**  Provide pressure-sensitive inputs, often used for actions like accelerating, braking, or firing.\n* **D-Pad:**  Dir

In [15]:
with_message_history.invoke(
    [HumanMessage(content="Explain first one")],
    config=config,
)

AIMessage(content='Let\'s break down the first feature: **User Interface**.  \n\nThink of the user interface (UI) of a controller as its "face" – the part you interact with directly. It\'s all about making the controller easy and intuitive to use.  \n\nHere\'s what makes a good controller UI:\n\n* **Clear and Organized:**  The buttons, joysticks, and other elements should be laid out in a logical way that\'s easy for users to understand at a glance. \n\n* **Tactile Feedback:**  \nButtons should provide a distinct "click" or "bump" when pressed, letting you know they\'ve been registered. This helps with accuracy and feedback during gameplay.\n\n* **Responsiveness:**  The controller should react quickly and accurately to user inputs. There shouldn\'t be any lag or delay between pressing a button and seeing the action on screen.\n* **Customizable:** Some controllers allow you to remap buttons or adjust the sensitivity of joysticks to suit your personal preferences. \n\nA well-designed UI 

In [11]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it! 😊\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! \n\nIt's nice to meet you. What can I do for you today? 😊  \n\n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"Your name is John, remember?  \n\nI'm happy to help you with anything else you need!  What can I do for you next? 😊 \n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In above case, the raw user input is just a message (Human message), which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking human messages as input). Next, we'll add in more input besides just the messages.

##### MessagePlaceholder
MessagesPlaceholder is a placeholder that tells LangChain where to insert past chat history when formatting a prompt.

Above RunnableWithMessageHistory only take chain(prompt+model) and history. 
- since we are using message structure (HumanMessage) that is why it does not require put input keys. 
- If there is only one replaceble parameter in prompt like messages(which represent chat history) then no need to explicitly declare chat history key while defining runnable parameter.
- Output from invoke is also a structured message (AIMessage)  that is why it does not require put output keys.

In [22]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages") # message represent chat history
    ]
)

chain=prompt|model

### Manually invoking without runnable

In [15]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]}) # messages var is replaced in template with value

AIMessage(content="Hello Krish! It's nice to meet you.\n\nI'm happy to help with any questions you have. Just ask away!  What can I do for you today? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 31, 'total_tokens': 73, 'completion_time': 0.076363636, 'prompt_time': 0.002332896, 'queue_time': 0.018076922999999998, 'total_time': 0.078696532}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3397d03b-8420-4fab-8a49-42e1a8fc6651-0', usage_metadata={'input_tokens': 31, 'output_tokens': 42, 'total_tokens': 73})

##### If there is only one replaceble parameter like messages then no need to explicitly declare it while defining runnable parameter.

In [23]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [24]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

AIMessage(content="Hi Krish! It's nice to meet you.  \n\nWhat can I help you with today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 270, 'total_tokens': 296, 'completion_time': 0.047272727, 'prompt_time': 0.012333677, 'queue_time': 0.019557949999999998, 'total_time': 0.059606404}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5ab8319a-e3a5-409c-acb5-672e185f2ade-0', usage_metadata={'input_tokens': 270, 'output_tokens': 26, 'total_tokens': 296})

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish!  \n\nI remember that from our introduction. 😊  How can I help you further?\n'

In [21]:
response = with_message_history.invoke(
    [HumanMessage(content="what is the first thing i said")],
    config=config,
)

response.content

'The first thing you said was "Hi My name is Krish".  \n\nDo you have any other questions for me? 😊  \n'

##### To increase complexity now we add 2 replaceable parameter in input
Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history. here we have specified key that contain input question

In [16]:
## Add more complexity
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

### Manually


In [17]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष्ण! 😊 \n\nमुझे बहुत खुशी है कि आपने मुझसे बात करना शुरू कर दी है। मैं आपकी मदद करने के लिए तैयार हूँ। \n\nआप मुझे क्या पूछना चाहेंगे? \n'

Above RunnableWithMessageHistory only take chain, input_key and history. 
- input is not message variable its a dict with keys (so need to tell clearly).
- input dictionary has messages key so no need of chat history key. 
- Output from invoke is also a structured message (AIMessage)  that is why it does not require put output keys.

In [28]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages" #we need to specify the correct key to use to save the chat history.
)

In [29]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

"Hello Krish! 👋 \n\nIt's nice to meet you.  How can I help you today? 😊  Ask me anything! \n\n"

In [30]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [31]:
response.content

'Your name is Krish! 😊  I remember that.\n\nIs there anything else I can help you with?\n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
- 'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages etc.

In [33]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last", # trim old memory first
    token_counter=model, # use LLM model for counter
    include_system=True, # include system msg
    allow_partial=False, # msg will not come partially if it is exceeding limit
    start_on="human"  # start trimming from human msg if needed.
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

a:\reacttutorial\reacttutorial\authorizeuser\LangChain\Memory component implementation\Venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

- itemgetter("messages"): Extracts the "messages" key from an input dictionary.
- RunnablePassthrough: A basic runnable that simply passes data through without modifications.

#### Using Chains for Automatic Trimming

In [34]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# temgetter("messages") extracts the "messages" key from the input dictionary.
# | trimmer then trims the messages
# The trimmed messages are assigned back to the "messages" key. with .assign

#New chain with trimmer
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content # reason: TRIMMED

"As a large language model, I don't have access to your personal information, including your favorite ice cream flavor.  \n\nWhat's your favorite ice cream flavor? 😊  \n\n"

In [37]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response

AIMessage(content='You asked "what is 2 + 2". 😊  \n\n\n\nWould you like to try another one?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 85, 'total_tokens': 110, 'completion_time': 0.045454545, 'prompt_time': 0.004059245, 'queue_time': 0.020894214, 'total_time': 0.04951379}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0ae74186-44cf-4dfb-a24b-287b496a599e-0', usage_metadata={'input_tokens': 85, 'output_tokens': 25, 'total_tokens': 110})

In [36]:
response.content

'You asked me "whats 2 + 2" 😊  \n\n\n\nLet me know if you have any other questions!\n'

In [32]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain, # CHAIN WITH TRIMMER
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [38]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="What ice cream do i like?")],
        "language": "English",
    },
    config=config,
)

response.content # out of context due to trim

"As a large language model, I don't have access to your personal information, so I don't know what your favorite ice cream flavor is.  \n\nWhat's your favorite flavor? Maybe I can tell you some fun facts about it! 🍦😊\n"

In [36]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked "whats 2 + 2"  😄 \n\n\n'